In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
# from optimum.gptq import GPTQQuantizer, load_quantized_model

In [21]:
!gpustat

gpu-server           Mon Feb  5 10:44:00 2024  495.29.05
[0] Quadro RTX 5000  | 28'C,   0 % |    17 / 16122 MB | gdm(9M) gdm(3M)


In [31]:
gpu_to_use = 0

# Set the device
device = torch.device(f'cuda:{gpu_to_use}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5",load_in_4bit = True
    # trust_remote_code=True,
    # cache_dir  = './tokenizer'
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5",load_in_4bit = True)

Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  3.00s/it]


In [23]:
def gen(prompt):
    toks = tokenizer(prompt,return_tensors = 'pt').to(device)
    output = model.generate(
        **toks,
        top_k = 1,
        do_sample = True,
        temperature = 0.2,
        max_new_tokens = 200,
        eos_token_id = [10897,63,4686,10252],
    )
    output = output[0][len(toks.input_ids[0]):]
    output = tokenizer.decode(output)
    return output
    

In [26]:
# prompt = """
# You're a highly skilled Python coder known for your mastery in pandas,seaborn,matplotlib,plotly.
# When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.
# Always import necessary libraries and load the dataset.
# ### Rules:
# 1) print values above the bars of bar plot.
# 2) the output should be phrased with respect to question.
# 3) Use seaborn or plotly along with matplotlib
# 4) do not print values above the bar if it gets too congested
# 5) only give the code without explanation.

# 6) I want you to generate what is asked, if i told you to read csv file the output should be:
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     import numpy as np            
#     df = pd.read_csv('Salaries.csv')
#     instead of :
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     import numpy as np
    
#     df = pd.read_csv('Salaries.csv')
    
#     # ln[10]:
#      python
#     plt.figure(figsize=(10,6))
#     sns.barplot(x='rank', y='salary', hue='discipline', data=df, palette='viridis')
# Generate what is asked, do not give extra output.


# You should strictly follow the above rule

# Here's the scoop on the "Salaries.csv" dataset, boasting 78 records featuring diverse columns:


# |   | rank | discipline | phd | service |  sex  | salary |
# |---|------|------------|-----|---------|-------|--------|
# | 0 | Prof |     B      | 56  |   49    | Male  | 186960 |
# | 1 | Prof |     A      | 12  |    6    | Male  |  93000 |
# | 2 | Prof |     A      | 23  |   20    | Male  | 110515 |
# | 3 | Prof |     A      | 40  |   31    | Male  | 131205 |
# | 4 | Prof |     B      | 20  |   18    | Male  | 104800 |

 
# ## majnu: {}
# ## Basha:

# # ln[3]:
# ``` python
# """

# nl_prompt =  prompt.format("create a bar plot for number of males and females")
# # nl_prompt =  prompt.format("Write a code to load a dataset")
# # print(f'prompt: {nl_prompt}')
# # print('-'*100)
# out = gen(nl_prompt).replace("```","")
# print(out)
# # print('-'*100)
# # exec(out)


In [34]:
prompt = """You're a highly skilled Python coder known for your mastery in pandas, seaborn, matplotlib, plotly.
When it comes to visualizations, your expertise shines through, crafting aesthetically pleasing plots with captivating color schemes.
Always import necessary libraries and load the dataset.

### Rules:
1) print values above the bars of bar plot.
2) the output should be phrased with respect to question.
3) Use seaborn or plotly along with matplotlib
4) do not print values above the bar if it gets too congested
5) only give the code without explanation and comments.

# You should strictly follow the above rule

Here's the scoop on the "Salaries.csv" dataset, boasting 78 records featuring diverse columns:

|   | rank | discipline | phd | service |  sex  | salary |
|---|------|------------|-----|---------|-------|--------|
| 0 | Prof |     B      | 56  |   49    | Male  | 186960 |
| 1 | Prof |     A      | 12  |    6    | Male  |  93000 |
| 2 | Prof |     A      | 23  |   20    | Male  | 110515 |
| 3 | Prof |     A      | 40  |   31    | Male  | 131205 |
| 4 | Prof |     B      | 20  |   18    | Male  | 104800 |

# Follow the example below
User: Give me salary distribution across all sex
Assistant: 
df = pd.read_csv("Salaries.csv")
salary_distribution = df.groupby('sex')['salary'].mean()
colors = ['skyblue', 'lightcoral']
salary_distribution.plot(kind='bar', color=colors)
plt.title('Salary Distribution Across Sexes')
plt.xlabel('Sex')
plt.ylabel('Average Salary')
plt.show()

# Assist user

User: {}
Assistant:
```python
df = pd.read_csv("Salaries.csv")"""

nl_prompt =  prompt.format("for each gender and discipline plot average salaries")
# nl_prompt =  prompt.format("Write a code to load a dataset")
# print(f'prompt: {nl_prompt}')
# print('-'*100)
out = gen(nl_prompt)
print(out)
# print('-'*100)
# exec(out)


Setting `pad_token_id` to `eos_token_id`:10897 for open-end generation.



pivot_table = df.pivot_table(values='salary', index='discipline', columns='sex', aggfunc='mean')
pivot_table.plot(kind='bar', stacked=True)
plt.title('Average Salaries by Discipline and Gender')
plt.xlabel('Discipline')
plt.ylabel('Average Salary')
plt.legend(title='Sex', loc='upper left')
plt.show()
```


In [22]:
tokenizer.convert_tokens_to_ids("```")

10897

In [32]:
df.columns



NameError: name 'df' is not defined